# 1. Importing Libraries

In [23]:
import pandas as pd
import numpy as np

import tensorflow as tf
from sklearn.model_selection import StratifiedKFold
from pytorch_tabnet.tab_model import TabNetClassifier

import os

### Testing Tensorflow GPU

In [24]:
tf.test.is_built_with_cuda()

True

# 2. Project Variables

In [25]:
DATA_DIR = '../train-test-data'
NUM_FOLDS = 10
TASKS_TO_RUN = ['2aii', '2aiii']

# 3. Model Training

In [26]:
# A utility method to create a tf.data dataset from a Pandas Dataframe
def df_to_dataset(dataframe, label, shuffle=True, batch_size=8):
    dataframe = dataframe.copy()
    dataframe['target'] = np.where(dataframe[label]=='INCREASED RISK', 1, 0)
    dataframe = dataframe.drop(columns=label)
    
    dataframe = dataframe.copy()
    labels = dataframe.pop('target')
    ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
    if shuffle:
        ds = ds.shuffle(buffer_size=len(dataframe))
        ds = ds.batch(batch_size)
    return ds

In [39]:
# A utitlity method to create an X and y np.array from a Pandas Dataframe
def df_to_nparray(dataframe, label):
    X = dataframe.drop(label, axis=1).to_numpy()
    y = dataframe[label].to_numpy()
    return X, y
    

In [51]:
# A utility method to train with k-fold
def train_kfold(num_fold, task, included_cols, train_func):
    # Read train csv
    train_df = pd.read_csv(os.path.join(DATA_DIR, f'{task}_train.csv'), index_col=0)

    # Metric arrays
    acc_per_fold = []
    loss_per_fold = []
    sens_per_fold = []
    spec_per_fold = []
    
    kfold = StratifiedKFold(n_splits=num_fold, shuffle=True, random_state=42)
    fold_no = 1
    for train_idx, val_idx in kfold.split(train_df.drop(task, axis=1), train_df[[task]]):
        train = train_df.iloc[train_idx] 
        test = train_df.iloc[val_idx]

        loss, accuracy, sensitivity, specificity = train_func(train,test, task)
        
        loss_per_fold.append(loss)
        acc_per_fold.append(accuracy)
        sens_per_fold.append(sensitivity)
        spec_per_fold.append(specificity)
        
        fold_no += 1
    
    metrics = {
        'ACCURACY': {
            'ALL': acc_per_fold,
            'MEAN': np.mean(acc_per_fold),
            'STDEV': np.std(acc_per_fold)
        },
        'SENSITIVITY': {
            'ALL': sens_per_fold,
            'MEAN': np.mean(sens_per_fold),
            'STDEV': np.std(sens_per_fold)
        },
        'SPECIFICITY': {
            'ALL': spec_per_fold,
            'MEAN': np.mean(spec_per_fold),
            'STDEV': np.std(spec_per_fold)
        }
    }
    return metrics

In [52]:
# Train neural network
def train_nn(train, test, task):
    # Generate feauture columns
    feature_columns = []
    for col in included_cols:
        feature_columns.append(tf.feature_column.numeric_column(col))

    # Generating a tensorflow dataset
    train_ds = df_to_dataset(train, task)
    test_ds = df_to_dataset(test, task)

    # Building model
    model = tf.keras.Sequential([
        tf.keras.layers.DenseFeatures(feature_columns),
        tf.keras.layers.Dense(14, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])

    model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy',
                           tf.keras.metrics.TruePositives(),
                           tf.keras.metrics.TrueNegatives(),
                           tf.keras.metrics.FalsePositives(),
                           tf.keras.metrics.FalseNegatives()
                          ])

    # Fitting Model
    history = model.fit(train_ds, epochs=10, verbose=1)

    # Evaluate Model
    scores = model.evaluate(test_ds, verbose=0)
    loss, accuracy, tp, tn, fp, fn = scores
    sensitivity = tp/(tp+fn)
    specificity = tn/(tn+fp)
    
    return loss, accuracy, sensitivity, specificity

In [53]:
def train_tabnet(train, test, task):
    X_train, y_train = df_to_nparray(train, task)
    X_test, y_test = df_to_nparray(test, task)

    model = TabNetClassifier()
    model.fit(X_train, y_train)
    preds = model.predict(X_test)

    tp, fp, tn, fn = [0,0,0,0]
    for p,a in zip(preds, y_test):
        if (p == 'INCREASED RISK' and a == 'INCREASED RISK'): tp += 1
        elif (p == 'INCREASED RISK' and a == 'REDUCED RISK'): fp += 1
        elif (p == 'REDUCED RISK' and a == 'INCREASED RISK'): fn += 1
        else: tn += 1

    accuracy = (tp+tn)/(tp+tn+fp+fn)
    sensitivity = tp/(tp+fn)
    specificity = tn/(tn+fp)

    return -1, accuracy, sensitivity, specificity

0.6732673267326733

In [54]:
metrics = {}

included_cols = ['CHILD_SEX','IDD_SCORE','AGE','HHID_count','HH_AGE','FOOD_EXPENSE_WEEKLY',
                 'NON-FOOD_EXPENSE_WEEKLY','HDD_SCORE','FOOD_INSECURITY','YoungBoys','YoungGirls',
                 'AverageMonthlyIncome','BEN_4PS','AREA_TYPE','FOOD_EXPENSE_WEEKLY_pc',
                 'NON-FOOD_EXPENSE_WEEKLY_pc','AverageMonthlyIncome_pc']

for task in TASKS_TO_RUN:
    metric = train_kfold(NUM_FOLDS, task, included_cols, train_tabnet)
    metrics[task] = metric

Device used : cpu
No early stopping will be performed, last training weights will be used.
epoch 0  | loss: 1.2214  |  0:00:00s
epoch 1  | loss: 0.91538 |  0:00:00s
epoch 2  | loss: 0.71774 |  0:00:00s
epoch 3  | loss: 0.60308 |  0:00:00s
epoch 4  | loss: 0.53341 |  0:00:00s
epoch 5  | loss: 0.47526 |  0:00:00s
epoch 6  | loss: 0.4647  |  0:00:00s
epoch 7  | loss: 0.45177 |  0:00:00s
epoch 8  | loss: 0.44853 |  0:00:00s
epoch 9  | loss: 0.45086 |  0:00:00s
epoch 10 | loss: 0.4426  |  0:00:00s
epoch 11 | loss: 0.44003 |  0:00:00s
epoch 12 | loss: 0.44779 |  0:00:00s
epoch 13 | loss: 0.44877 |  0:00:00s
epoch 14 | loss: 0.45707 |  0:00:00s
epoch 15 | loss: 0.43454 |  0:00:00s
epoch 16 | loss: 0.42927 |  0:00:00s
epoch 17 | loss: 0.41648 |  0:00:00s
epoch 18 | loss: 0.43763 |  0:00:00s
epoch 19 | loss: 0.41593 |  0:00:00s
epoch 20 | loss: 0.41877 |  0:00:00s
epoch 21 | loss: 0.39907 |  0:00:00s
epoch 22 | loss: 0.39011 |  0:00:00s
epoch 23 | loss: 0.3869  |  0:00:00s
epoch 24 | loss: 0.36

epoch 16 | loss: 0.37639 |  0:00:00s
epoch 17 | loss: 0.38371 |  0:00:00s
epoch 18 | loss: 0.36964 |  0:00:00s
epoch 19 | loss: 0.36377 |  0:00:00s
epoch 20 | loss: 0.35283 |  0:00:00s
epoch 21 | loss: 0.36079 |  0:00:00s
epoch 22 | loss: 0.34357 |  0:00:00s
epoch 23 | loss: 0.36675 |  0:00:00s
epoch 24 | loss: 0.3477  |  0:00:00s
epoch 25 | loss: 0.33946 |  0:00:00s
epoch 26 | loss: 0.35364 |  0:00:00s
epoch 27 | loss: 0.34608 |  0:00:00s
epoch 28 | loss: 0.32242 |  0:00:00s
epoch 29 | loss: 0.33781 |  0:00:00s
epoch 30 | loss: 0.31921 |  0:00:00s
epoch 31 | loss: 0.3355  |  0:00:00s
epoch 32 | loss: 0.3181  |  0:00:00s
epoch 33 | loss: 0.3314  |  0:00:00s
epoch 34 | loss: 0.32436 |  0:00:00s
epoch 35 | loss: 0.29872 |  0:00:01s
epoch 36 | loss: 0.33706 |  0:00:01s
epoch 37 | loss: 0.30088 |  0:00:01s
epoch 38 | loss: 0.31118 |  0:00:01s
epoch 39 | loss: 0.31549 |  0:00:01s
epoch 40 | loss: 0.31002 |  0:00:01s
epoch 41 | loss: 0.29946 |  0:00:01s
epoch 42 | loss: 0.30898 |  0:00:01s
e

epoch 39 | loss: 0.30398 |  0:00:01s
epoch 40 | loss: 0.29411 |  0:00:01s
epoch 41 | loss: 0.27785 |  0:00:01s
epoch 42 | loss: 0.29185 |  0:00:01s
epoch 43 | loss: 0.28506 |  0:00:01s
epoch 44 | loss: 0.26175 |  0:00:01s
epoch 45 | loss: 0.27887 |  0:00:01s
epoch 46 | loss: 0.28162 |  0:00:01s
epoch 47 | loss: 0.26422 |  0:00:01s
epoch 48 | loss: 0.24708 |  0:00:01s
epoch 49 | loss: 0.27185 |  0:00:01s
epoch 50 | loss: 0.25664 |  0:00:01s
epoch 51 | loss: 0.23555 |  0:00:01s
epoch 52 | loss: 0.25139 |  0:00:01s
epoch 53 | loss: 0.24663 |  0:00:01s
epoch 54 | loss: 0.26331 |  0:00:01s
epoch 55 | loss: 0.2371  |  0:00:01s
epoch 56 | loss: 0.24497 |  0:00:01s
epoch 57 | loss: 0.23982 |  0:00:01s
epoch 58 | loss: 0.24118 |  0:00:01s
epoch 59 | loss: 0.22603 |  0:00:01s
epoch 60 | loss: 0.23033 |  0:00:01s
epoch 61 | loss: 0.21811 |  0:00:01s
epoch 62 | loss: 0.22226 |  0:00:01s
epoch 63 | loss: 0.22423 |  0:00:01s
epoch 64 | loss: 0.19946 |  0:00:01s
epoch 65 | loss: 0.2236  |  0:00:01s
e

epoch 60 | loss: 0.20602 |  0:00:01s
epoch 61 | loss: 0.22271 |  0:00:01s
epoch 62 | loss: 0.25368 |  0:00:01s
epoch 63 | loss: 0.20384 |  0:00:01s
epoch 64 | loss: 0.21381 |  0:00:01s
epoch 65 | loss: 0.21299 |  0:00:01s
epoch 66 | loss: 0.22798 |  0:00:01s
epoch 67 | loss: 0.20967 |  0:00:01s
epoch 68 | loss: 0.22655 |  0:00:01s
epoch 69 | loss: 0.1834  |  0:00:01s
epoch 70 | loss: 0.20265 |  0:00:01s
epoch 71 | loss: 0.17985 |  0:00:01s
epoch 72 | loss: 0.20444 |  0:00:01s
epoch 73 | loss: 0.17386 |  0:00:01s
epoch 74 | loss: 0.19254 |  0:00:01s
epoch 75 | loss: 0.1973  |  0:00:01s
epoch 76 | loss: 0.18944 |  0:00:02s
epoch 77 | loss: 0.14782 |  0:00:02s
epoch 78 | loss: 0.14767 |  0:00:02s
epoch 79 | loss: 0.15531 |  0:00:02s
epoch 80 | loss: 0.19942 |  0:00:02s
epoch 81 | loss: 0.23667 |  0:00:02s
epoch 82 | loss: 0.11894 |  0:00:02s
epoch 83 | loss: 0.15078 |  0:00:02s
epoch 84 | loss: 0.12805 |  0:00:02s
epoch 85 | loss: 0.13914 |  0:00:02s
epoch 86 | loss: 0.12305 |  0:00:02s
e

epoch 80 | loss: 0.18002 |  0:00:02s
epoch 81 | loss: 0.22192 |  0:00:02s
epoch 82 | loss: 0.21854 |  0:00:02s
epoch 83 | loss: 0.19191 |  0:00:02s
epoch 84 | loss: 0.19543 |  0:00:02s
epoch 85 | loss: 0.21135 |  0:00:02s
epoch 86 | loss: 0.20053 |  0:00:02s
epoch 87 | loss: 0.22186 |  0:00:02s
epoch 88 | loss: 0.2156  |  0:00:02s
epoch 89 | loss: 0.1954  |  0:00:02s
epoch 90 | loss: 0.17922 |  0:00:02s
epoch 91 | loss: 0.1765  |  0:00:02s
epoch 92 | loss: 0.22043 |  0:00:02s
epoch 93 | loss: 0.19152 |  0:00:02s
epoch 94 | loss: 0.16126 |  0:00:02s
epoch 95 | loss: 0.16295 |  0:00:02s
epoch 96 | loss: 0.162   |  0:00:02s
epoch 97 | loss: 0.18415 |  0:00:02s
epoch 98 | loss: 0.158   |  0:00:02s
epoch 99 | loss: 0.19899 |  0:00:02s
Device used : cpu
No early stopping will be performed, last training weights will be used.
epoch 0  | loss: 1.14388 |  0:00:00s
epoch 1  | loss: 0.89723 |  0:00:00s
epoch 2  | loss: 0.69746 |  0:00:00s
epoch 3  | loss: 0.6257  |  0:00:00s
epoch 4  | loss: 0.54

epoch 97 | loss: 0.22683 |  0:00:02s
epoch 98 | loss: 0.18218 |  0:00:02s
epoch 99 | loss: 0.19652 |  0:00:02s
Device used : cpu
No early stopping will be performed, last training weights will be used.
epoch 0  | loss: 1.172   |  0:00:00s
epoch 1  | loss: 0.87257 |  0:00:00s
epoch 2  | loss: 0.72271 |  0:00:00s
epoch 3  | loss: 0.59716 |  0:00:00s
epoch 4  | loss: 0.58759 |  0:00:00s
epoch 5  | loss: 0.54999 |  0:00:00s
epoch 6  | loss: 0.52267 |  0:00:00s
epoch 7  | loss: 0.50586 |  0:00:00s
epoch 8  | loss: 0.53501 |  0:00:00s
epoch 9  | loss: 0.5138  |  0:00:00s
epoch 10 | loss: 0.46964 |  0:00:00s
epoch 11 | loss: 0.47103 |  0:00:00s
epoch 12 | loss: 0.47663 |  0:00:00s
epoch 13 | loss: 0.46633 |  0:00:00s
epoch 14 | loss: 0.46759 |  0:00:00s
epoch 15 | loss: 0.47242 |  0:00:00s
epoch 16 | loss: 0.46326 |  0:00:00s
epoch 17 | loss: 0.47814 |  0:00:00s
epoch 18 | loss: 0.4524  |  0:00:00s
epoch 19 | loss: 0.45614 |  0:00:00s
epoch 20 | loss: 0.44831 |  0:00:00s
epoch 21 | loss: 0.43

epoch 17 | loss: 0.44419 |  0:00:00s
epoch 18 | loss: 0.431   |  0:00:00s
epoch 19 | loss: 0.44782 |  0:00:00s
epoch 20 | loss: 0.4242  |  0:00:00s
epoch 21 | loss: 0.44444 |  0:00:00s
epoch 22 | loss: 0.44009 |  0:00:00s
epoch 23 | loss: 0.42399 |  0:00:00s
epoch 24 | loss: 0.42752 |  0:00:00s
epoch 25 | loss: 0.42908 |  0:00:00s
epoch 26 | loss: 0.40942 |  0:00:00s
epoch 27 | loss: 0.39835 |  0:00:00s
epoch 28 | loss: 0.40739 |  0:00:00s
epoch 29 | loss: 0.38986 |  0:00:00s
epoch 30 | loss: 0.3954  |  0:00:00s
epoch 31 | loss: 0.38535 |  0:00:00s
epoch 32 | loss: 0.3909  |  0:00:00s
epoch 33 | loss: 0.37537 |  0:00:00s
epoch 34 | loss: 0.39894 |  0:00:00s
epoch 35 | loss: 0.38761 |  0:00:00s
epoch 36 | loss: 0.34671 |  0:00:00s
epoch 37 | loss: 0.37642 |  0:00:01s
epoch 38 | loss: 0.36799 |  0:00:01s
epoch 39 | loss: 0.37477 |  0:00:01s
epoch 40 | loss: 0.35551 |  0:00:01s
epoch 41 | loss: 0.3664  |  0:00:01s
epoch 42 | loss: 0.37406 |  0:00:01s
epoch 43 | loss: 0.34447 |  0:00:01s
e

epoch 38 | loss: 0.4047  |  0:00:01s
epoch 39 | loss: 0.39923 |  0:00:01s
epoch 40 | loss: 0.38749 |  0:00:01s
epoch 41 | loss: 0.38736 |  0:00:01s
epoch 42 | loss: 0.41259 |  0:00:01s
epoch 43 | loss: 0.39071 |  0:00:01s
epoch 44 | loss: 0.38804 |  0:00:01s
epoch 45 | loss: 0.37909 |  0:00:01s
epoch 46 | loss: 0.38868 |  0:00:01s
epoch 47 | loss: 0.36714 |  0:00:01s
epoch 48 | loss: 0.36451 |  0:00:01s
epoch 49 | loss: 0.37248 |  0:00:01s
epoch 50 | loss: 0.36943 |  0:00:01s
epoch 51 | loss: 0.40854 |  0:00:01s
epoch 52 | loss: 0.34882 |  0:00:01s
epoch 53 | loss: 0.36423 |  0:00:01s
epoch 54 | loss: 0.34542 |  0:00:01s
epoch 55 | loss: 0.36567 |  0:00:01s
epoch 56 | loss: 0.32572 |  0:00:01s
epoch 57 | loss: 0.36903 |  0:00:01s
epoch 58 | loss: 0.33391 |  0:00:01s
epoch 59 | loss: 0.2989  |  0:00:01s
epoch 60 | loss: 0.30131 |  0:00:01s
epoch 61 | loss: 0.39298 |  0:00:01s
epoch 62 | loss: 0.30661 |  0:00:01s
epoch 63 | loss: 0.29269 |  0:00:01s
epoch 64 | loss: 0.30603 |  0:00:01s
e

epoch 57 | loss: 0.3807  |  0:00:01s
epoch 58 | loss: 0.35363 |  0:00:01s
epoch 59 | loss: 0.3213  |  0:00:01s
epoch 60 | loss: 0.35385 |  0:00:01s
epoch 61 | loss: 0.31994 |  0:00:01s
epoch 62 | loss: 0.33406 |  0:00:01s
epoch 63 | loss: 0.3214  |  0:00:01s
epoch 64 | loss: 0.30831 |  0:00:01s
epoch 65 | loss: 0.35709 |  0:00:01s
epoch 66 | loss: 0.30958 |  0:00:01s
epoch 67 | loss: 0.29879 |  0:00:01s
epoch 68 | loss: 0.32942 |  0:00:01s
epoch 69 | loss: 0.29264 |  0:00:01s
epoch 70 | loss: 0.29327 |  0:00:01s
epoch 71 | loss: 0.26783 |  0:00:01s
epoch 72 | loss: 0.30436 |  0:00:01s
epoch 73 | loss: 0.35048 |  0:00:01s
epoch 74 | loss: 0.31368 |  0:00:01s
epoch 75 | loss: 0.33215 |  0:00:02s
epoch 76 | loss: 0.30384 |  0:00:02s
epoch 77 | loss: 0.3216  |  0:00:02s
epoch 78 | loss: 0.29061 |  0:00:02s
epoch 79 | loss: 0.3288  |  0:00:02s
epoch 80 | loss: 0.30081 |  0:00:02s
epoch 81 | loss: 0.32457 |  0:00:02s
epoch 82 | loss: 0.3124  |  0:00:02s
epoch 83 | loss: 0.25575 |  0:00:02s
e

epoch 77 | loss: 0.23618 |  0:00:01s
epoch 78 | loss: 0.27278 |  0:00:02s
epoch 79 | loss: 0.25246 |  0:00:02s
epoch 80 | loss: 0.22864 |  0:00:02s
epoch 81 | loss: 0.24284 |  0:00:02s
epoch 82 | loss: 0.22272 |  0:00:02s
epoch 83 | loss: 0.24044 |  0:00:02s
epoch 84 | loss: 0.24995 |  0:00:02s
epoch 85 | loss: 0.2082  |  0:00:02s
epoch 86 | loss: 0.21936 |  0:00:02s
epoch 87 | loss: 0.21863 |  0:00:02s
epoch 88 | loss: 0.24448 |  0:00:02s
epoch 89 | loss: 0.25571 |  0:00:02s
epoch 90 | loss: 0.21951 |  0:00:02s
epoch 91 | loss: 0.19346 |  0:00:02s
epoch 92 | loss: 0.24176 |  0:00:02s
epoch 93 | loss: 0.26809 |  0:00:02s
epoch 94 | loss: 0.20535 |  0:00:02s
epoch 95 | loss: 0.20927 |  0:00:02s
epoch 96 | loss: 0.19486 |  0:00:02s
epoch 97 | loss: 0.18917 |  0:00:02s
epoch 98 | loss: 0.19364 |  0:00:02s
epoch 99 | loss: 0.22146 |  0:00:02s


In [55]:
for task in TASKS_TO_RUN:
    print(f'{task}: ACCURACY: {metrics[task]["ACCURACY"]["MEAN"]} SENSITIVITY: {metrics[task]["SENSITIVITY"]["MEAN"]} SPECIFICITY: {metrics[task]["SPECIFICITY"]["MEAN"]}')

2aii: ACCURACY: 0.7493548387096773 SENSITIVITY: 0.8687692307692307 SPECIFICITY: 0.17333333333333334
2aiii: ACCURACY: 0.73247311827957 SENSITIVITY: 0.9034420289855072 SPECIFICITY: 0.10714285714285714


In [8]:
metrics['2aii']['ACCURACY']['ALL']

[61.29032373428345,
 67.7419364452362,
 74.19354915618896,
 69.9999988079071,
 73.33333492279053,
 80.0000011920929,
 80.0000011920929,
 73.33333492279053,
 83.33333134651184,
 83.33333134651184]

# Model Evaluation

### Note: This runs evaluates the models with the testing set. Run only at the end.

In [87]:
def train_and_test(task, included_cols, models):
    train_df = pd.read_csv(os.path.join(DATA_DIR, f'{task}_train.csv'), index_col=0)
    test_df = pd.read_csv(os.path.join(DATA_DIR, f'{task}_test.csv'), index_col=0)
    
    # Generate feauture columns
    feature_columns = []
    for col in included_cols:
        feature_columns.append(tf.feature_column.numeric_column(col))

    # Generating a tensorflow dataset
    train_ds = df_to_dataset(train_df, task)
    test_ds = df_to_dataset(test_df, task)

    # Building model
    models[task] = tf.keras.Sequential([
        tf.keras.layers.DenseFeatures(feature_columns),
        tf.keras.layers.Dense(14, activation='relu'),
        tf.keras.layers.Dense(14, activation='relu'),
        tf.keras.layers.Dense(14, activation='relu'),
        tf.keras.layers.Dense(14, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])

    models[task].compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy',
                           tf.keras.metrics.TruePositives(),
                           tf.keras.metrics.TrueNegatives(),
                           tf.keras.metrics.FalsePositives(),
                           tf.keras.metrics.FalseNegatives()
                          ])

    # Fitting Model
    history = models[task].fit(train_ds, 
                        epochs=10, 
                        verbose=1)

    # Evaluate Model
    scores = models[task].evaluate(test_ds, verbose=0)
    tp, tn, fp, fn = scores[2:]
    sensitivity = tp/(tp+fn)
    specificity = tn/(tn+fp)

    
    metrics = {
        'ACCURACY': scores[1]*100,
        'SENSITIVITY': sensitivity,
        'SPECIFICITY': specificity
    }
    return metrics

In [88]:
models = {}
metrics = {}
included_cols = ['CHILD_SEX','IDD_SCORE','AGE','HHID_count','HH_AGE','FOOD_EXPENSE_WEEKLY',
                 'NON-FOOD_EXPENSE_WEEKLY','HDD_SCORE','FOOD_INSECURITY','YoungBoys','YoungGirls',
                 'AverageMonthlyIncome','BEN_4PS','AREA_TYPE','FOOD_EXPENSE_WEEKLY_pc',
                 'NON-FOOD_EXPENSE_WEEKLY_pc','AverageMonthlyIncome_pc']

for task in TASKS_TO_RUN:
    metric = train_and_test(task, included_cols, models)
    metrics[task] = metric

Epoch 1/10
38/38 [==============================] - 1s 8ms/step - loss: 19.6674 - accuracy: 0.7591 - true_positives_50: 225.0000 - true_negatives_25: 5.0000 - false_positives_25: 47.0000 - false_negatives_25: 26.0000  
Epoch 2/10
38/38 [==============================] - 0s 7ms/step - loss: 3.8524 - accuracy: 0.7492 - true_positives_50: 223.0000 - true_negatives_25: 4.0000 - false_positives_25: 48.0000 - false_negatives_25: 28.0000
Epoch 3/10
38/38 [==============================] - 0s 7ms/step - loss: 2.4745 - accuracy: 0.7294 - true_positives_50: 217.0000 - true_negatives_25: 4.0000 - false_positives_25: 48.0000 - false_negatives_25: 34.0000
Epoch 4/10
38/38 [==============================] - 0s 7ms/step - loss: 1.6355 - accuracy: 0.7822 - true_positives_50: 231.0000 - true_negatives_25: 6.0000 - false_positives_25: 46.0000 - false_negatives_25: 20.0000
Epoch 5/10
38/38 [==============================] - 0s 7ms/step - loss: 1.5104 - accuracy: 0.7327 - true_positives_50: 217.0000 - tru

38/38 [==============================] - 1s 7ms/step - loss: 18.4594 - accuracy: 0.7591 - true_positives_51: 228.0000 - true_negatives_26: 2.0000 - false_positives_26: 63.0000 - false_negatives_26: 10.0000 
Epoch 2/10
38/38 [==============================] - 0s 6ms/step - loss: 4.8151 - accuracy: 0.7129 - true_positives_51: 207.0000 - true_negatives_26: 9.0000 - false_positives_26: 56.0000 - false_negatives_26: 31.0000
Epoch 3/10
38/38 [==============================] - 0s 7ms/step - loss: 2.0059 - accuracy: 0.6799 - true_positives_51: 190.0000 - true_negatives_26: 16.0000 - false_positives_26: 49.0000 - false_negatives_26: 48.0000
Epoch 4/10
38/38 [==============================] - 0s 7ms/step - loss: 1.4519 - accuracy: 0.6700 - true_positives_51: 184.0000 - true_negatives_26: 19.0000 - false_positives_26: 46.0000 - false_negatives_26: 54.0000
Epoch 5/10
38/38 [==============================] - 0s 7ms/step - loss: 1.8110 - accuracy: 0.7030 - true_positives_51: 208.0000 - true_negative

In [89]:
for task in TASKS_TO_RUN:
    print(f'{task} - ACCURACY: {metrics[task]["ACCURACY"]} SENSITIVITY: {metrics[task]["SENSITIVITY"]} SPECIFICITY: {metrics[task]["SPECIFICITY"]}')

2aii - ACCURACY: 64.68647122383118 SENSITIVITY: 0.7 SPECIFICITY: 0.39622641509433965
2aiii - ACCURACY: 74.25742745399475 SENSITIVITY: 0.9079497907949791 SPECIFICITY: 0.125
